In [8]:
# !pip install fancyimpute
import numpy as np
import pandas as pd
from fancyimpute import IterativeImputer
from sklearn.preprocessing import StandardScaler

In [9]:
'''
pre-processes melanoma data

'''

'\npre-processes melanoma data\n\n'

In [46]:
# read in data

raw_data = pd.read_csv('../Melanoma/total_PD_1.csv')
raw_data.tail()

/var/folders/cn/ccmq_jg52yxblqm5cp00kr400000gn/T/ipykernel_61346/107892282.py:3: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('../Melanoma/total_PD_1.csv')


,Unnamed: 0,GSM2445716,GSM2445717,GSM2445718,GSM2445719,GSM2445720,GSM2445721,GSM2445722,GSM2445723,GSM2445724,...,SRR5088909,SRR5088911,SRR5088913,SRR5088914,SRR5088916,SRR5088920,SRR5088922,SRR5088924,SRR5088926,SRR5088929
56265,snoZ6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56266,snosnR66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56267,uc_338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.003664563208162,0.031978130771298,0.0857459707118245,0.04071053428664,0.0512846604842121,0.158733159882816,0.18278951659712,0.0657273971922831,0.0777414803400673,0.0
56268,yR211F11.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0355281824962792,0.0
56269,Response,N,N,N,N,N,N,N,N,N,...,N,R,N,N,N,N,N,R,N,R


In [70]:
# some re-formatting

# rename column

raw_data = raw_data.rename(columns={raw_data.columns[0]: 'patient'})
print(raw_data.shape)
#drop GSE931
raw_data_drop = raw_data.drop(raw_data.columns[raw_data.columns.str.contains("GSM24457")], axis=1)
print(raw_data_drop.shape)
# transpose so columns are genes and rows are patients
t_data = raw_data_drop.set_index('patient').T
# t_data = raw_data.set_index('patient').T
print(t_data.shape)

# drop rows(patient) with resopnse = UNK

t_data = t_data[t_data["Response"] != "UNK"]
print(t_data.shape)
# split data

labels = t_data['Response']
clean_data = t_data.drop(columns = ['Response'])


(56270, 158)
(56270, 133)
(132, 56270)
(125, 56270)


In [71]:
# drop any genes with > 90% missing data

clean_data = clean_data.dropna(thresh = clean_data.shape[0]*0.9, axis=1)

# drop genes with >90% 0 input 
zero_percent = (clean_data == 0). mean(axis = 0)
clean_data = clean_data.loc[:,zero_percent<0.9]
print(clean_data.shape)

(125, 40765)


In [72]:
# log2 transformation

# note that samples from the Melanoma dataset GSE931 are already log2 transformed

# add 1 to all values for log2 transformation

clean_data = clean_data.applymap(lambda x: x+1 if isinstance(x, (int, float)) else x)

# subset GSE931 samples from rest
# gse931 = clean_data[clean_data.index.str.contains('GSM24457')]

# to_transform = clean_data[~clean_data.index.str.contains("GSM24457")]

# log2 transform
to_transform = clean_data
log2_data = to_transform.applymap(lambda x: np.log2(x) if isinstance(x, (int, float)) else x)

# re-join data

# log2_data = pd.concat([gse931, log2_data], axis = 0)
log2_data.head()


/var/folders/cn/ccmq_jg52yxblqm5cp00kr400000gn/T/ipykernel_61346/1426497560.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clean_data = clean_data.applymap(lambda x: x+1 if isinstance(x, (int, float)) else x)
/var/folders/cn/ccmq_jg52yxblqm5cp00kr400000gn/T/ipykernel_61346/1426497560.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  log2_data = to_transform.applymap(lambda x: np.log2(x) if isinstance(x, (int, float)) else x)


patient,5S_rRNA,7SK,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,...,snoU2-30,snoU2_19,snoU83B,snoZ196,snoZ278,snoZ40,snoZ6,snosnR66,uc_338,yR211F11.2
SRR7344546,0.000000,1.164935,0.177984,0.646144,0.051743,7.388509,1.137378,0.051041,0.129775,0.000000,...,3.13092825610652,3.28747466891185,0.0,0.0,0.0,0.0,0.0,0.0,0.416319623212903,0.0
SRR7344554,0.128807,1.358727,0.070845,0.784005,0.015067,7.796801,2.642253,0.221139,0.119809,0.135597,...,13.6818810287859,1.57521656581416,0.0,0.0,0.0,0.0,0.0,0.0,0.364929374639851,0.0
SRR7344556,0.162194,0.143445,0.000000,0.000000,0.012671,4.623303,1.391299,0.016639,0.306234,0.114778,...,4.23564377253204,0.264727735783252,0.0,0.0,0.0,0.0,0.0,0.0,0.487394747280299,0.0
SRR7344564,0.267733,0.030576,0.326047,2.220248,0.105718,7.145439,0.736532,0.055363,0.562818,0.044087,...,0.226744499164846,0.0,0.360729885034982,2.85341392207446,0.0,0.0,0.0,0.0,0.0747493692947006,0.0
SRR7344565,0.135665,0.053568,0.241141,1.811070,0.097685,6.680064,0.427896,0.927511,0.202232,0.076971,...,0.0,0.437981509864241,0.0,1.57476497928716,0.0,0.0,0.0,0.0,0.0583043344721167,0.0


In [73]:
# z-score transformation (for PCA)

scaler = StandardScaler()
scaled_data = scaler.fit_transform(log2_data)

scaled_data = pd.DataFrame(scaled_data)
scaled_data.head()
print(scaled_data.shape)

(125, 40765)


In [75]:
# export '/content/total_PD_1.csv'

# imputed log2 transformed data
# imputed_data.to_csv('imputed_data_v3.csv')

#log2 transformed
log2_data.to_csv('log2_data_v3.csv')

# normalized version of above^ (for PCA)
scaled_data.to_csv('scaled_data_v3.csv')

# true labels (response y/n)
labels.to_csv('labels_v3.csv')